# 프로젝트: ResNet Ablation Study

In [18]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
from keras.initializers import glorot_uniform
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요. 
print(ds_info.features)

FeaturesDict({
    'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})


In [4]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

tf.Tensor(-2, shape=(), dtype=int64)
tf.Tensor(-2, shape=(), dtype=int64)


In [5]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [6]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

## ResNet 기본 블록 구성하기

In [40]:
# function for building VGG Block

# block_num : 레이어 이름 붙이기 위해
# is_50 ResNet-50인지 구분하기 위해
def build_resnet_block(input_layer,
                    num_cnn=3, 
                    channel=64,
                    block_num=1,
                    is_50 = False
                   ):    
    # 입력 레이어 , 추후 shortcut 에 사용
    x = input_layer

    # CNN 레이어    
    for cnn_num in range(num_cnn):
       
        #ResNet-50
        if is_50:
            # 1x1 layer
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(1,1),
                activation='relu',
                kernel_initializer=glorot_uniform(seed=0),
                name=f'block{block_num}_1x1_1'
            )(x)        

            x = keras.layers.BatchNormalization(axis = 3, name=f'block{block_num}_bn_1')(x)
            x = keras.layers.Activation('relu')(x)
            
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(3,3),
                activation='relu',
                kernel_initializer=glorot_uniform(seed=0),            
                name=f'block{block_num}_conv_{cnn_num}'
            )(x) 

            x = keras.layers.BatchNormalization(axis = 3, name=f'block{block_num}_bn_2')(x)
            x = keras.layers.Activation('relu')(x)
            
            # 1x1 layer
            x = keras.layers.Conv2D(
                filters=channel*4,
                kernel_size=(1,1),
                activation='relu',
                kernel_initializer=glorot_uniform(seed=0),
                name=f'block{block_num}_1x1_2'
            )(x)

            x = keras.layers.BatchNormalization(axis = 3, name=f'block{block_num}_bn_3')(x)

        #ResNet-34
        else:
            print('========== ResNet-34 Block_{}_{} =============='.format(block_num,cnn_num))
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(3,3),
                activation='relu',
                kernel_initializer=glorot_uniform(seed=0),
                name=f'block{block_num}_conv_{cnn_num}_1'
            )(x) 

            x = keras.layers.BatchNormalization(axis = 3, name=f'block{block_num}_bn_1')(x)
            x = keras.layers.Activation('relu')(x)

            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(3,3),
                activation='relu',
                kernel_initializer=glorot_uniform(seed=0),
                name=f'block{block_num}_conv_{cnn_num}_2'
            )(x) 

            x = keras.layers.BatchNormalization(axis = 3, name=f'block{block_num}_bn_2')(x)
    
        # shortcut 형태맞추기            
        x_shorcut = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            kernel_initializer=glorot_uniform(seed=0),
            name=f'shorcut'
        )(input_layer)
        
        x_shortcut = keras.layers.BatchNormalization(axis=3,name='shortcut_bn')(x_shorcut)

        x = keras.layers.Add()([x,x_shortcut])
        x = keras.layers.Activation('relu')(x)

    return x

## ResNet-34, ResNet-50 Complete Model

In [41]:
from keras.initializers import glorot_uniform
def build_resnet(input_shape=(32,32,3),is_50 = False, num_classes=1000):

    num_cnn_list = [3,4,6,3]
    channel_list = [64,128,256,512]
    num_classes = num_classes

    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    output = keras.layers.ZeroPadding2D((3,3))(input_layer)

    # conv1 7x7 64, stride 2
    output = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides = (2,2),
        activation='relu',
        # 인터넷 참조
        kernel_initializer=glorot_uniform(seed=0),    
        name ='conv1'
    )(output)

    # BN
    output = keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(output)
    # Activation _ ReLu
    output = keras.layers.Activation('relu')(output)     

    # 3x3 max pool, stride 2
    output = keras.layers.MaxPooling2D(
        pool_size=(3, 3),
        strides=2,
        name=f'conv2_max_pool'
    )(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block(
            output,
            num_cnn=num_cnn, 
            channel=channel,            
            block_num=i,
            is_50 = is_50
        )

    output = keras.layers.AveragePooling2D(
        name=f'average_pool'
    )(output)
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='fc', kernel_initializer= glorot_uniform(seed=0))(output)    

    model = keras.Model(
        inputs = input_layer,
        outpus = output
    )

    return model


In [42]:
resnet_34 = build_resnet(input_shape=(32,32,3),is_50 = False)
resnet_34.summary()

========== ResNet-34 Block_0_0 ==============


ValueError: Operands could not be broadcast together with shapes (3, 3, 64) (7, 7, 64)

## 일반 네트워크(plain network) 만들기

## ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34


### cat_vs_dogs 로 비교해보기